# Exercise 5: Blocks and inlined rules

This notebook introduces more complex language structures that are able to restrict the matching context or even extend the expressivity of the matching process. 

In [ ]:
%%documentText
Peter works for Frank.
10€ are less than 100$.

We annotate all pairs of tokens within sentences (not across sentences) using a BLOCK construct. BLOCKs can also be used as an if statement, as a loop (including recursion), as named procedures and more.

In [ ]:
%displayMode CSV
%csvConfig TokenPair1

DECLARE Sentence;
(ANY+{-PARTOF(Sentence),-PARTOF(PERIOD)} PERIOD){-> Sentence};

DECLARE TokenPair1;

BLOCK(sentence) Sentence{}{
    (ANY ANY){-> TokenPair1};
}

We do the same using an inlined rule as action. Inlined rules as action indicated by "->" after the rule element can apply additional rules within the scope of the match of a rule element if the complete rule matched successfully.

In [ ]:
%csvConfig TokenPair2

DECLARE TokenPair2;

Sentence->{
    (ANY ANY){-> TokenPair2};
};

In the next example, we use the block like an if statement.

In [ ]:
%csvConfig TokenPair2

DECLARE TokenPair3;

// only in documents that contain numbers
BLOCK(sentence) Document{CONTAINS(NUM)}{
    Sentence->{
        (ANY ANY){-> TokenPair3};
    };
}

Next, we annotate all sentences that contain an WorksFor relation with an employee named "Peter". Complex conditions like these can be specified using inlined rules as condition. Rule elements extended with these inlined rules indicated with "<-" only match if at least one rule within the inlined block is able to match.

In [ ]:
%csvConfig SentenceWithRelation
DECLARE Employer, Employee;
"Peter"-> Employee;
"Frank"-> Employer;

DECLARE WorksFor (Employee employee, Employer employer);
(e1:Employee # e2:Employer){-> wf:WorksFor, wf.employee=e1, wf.employer=e2};

DECLARE SentenceWithRelation;

Sentence{-> SentenceWithRelation}
    <-{WorksFor.employee.ct=="Peter";};


Now, we annotate all sentences that are followed by a sentence that contains a MoneyAmount in Euro. These inlined rules can of course also be used in sequential patterns and they can also be used within other inlined rule blocks. Multiple inlined rule blocks at one rule element are also supported. If multiple inlined rule as condition block are given at one rule element at least one rule in each of them needs to be able to match, i.e. they specify a conjunction.

In [ ]:
%csvConfig SentenceFollowedByMoneySentence
DECLARE MoneyAmount(INT amount, STRING currency);
INT value;
(NUM{PARSE(value)} c:SPECIAL){-> CREATE(MoneyAmount, "amount"=value, "currency"=c.ct)};

DECLARE SentenceFollowedByMoneySentence;

Sentence{-> SentenceFollowedByMoneySentence} Sentence<-{MoneyAmount.currency.ct=="€";};


We annotate each word that is either followed or preceded by a small-written word using a FOREACH block. The FOREACH block is a special block, which assigns the annotation matched by the head rule to a temporary variables. This annoations variable can be used with the block. The FOREACH block does not restrict the window for the contained rules. The FOREACH block is very similar to a loop over a defined set of annotations. It iterates from left to right, but can also iterate from right to left given an additional argument. This block constuct can be used for more efficient rule executions (like a simple FST automata) since the variable can be matched directly without iterating over the index anew.

In [ ]:
%displayMode RUTA_COLORING

DECLARE Word;

FOREACH(w) W{}{
    w{->Word} SW;
    SW w{-> Word};
}

COLOR(Word, "pink");
